чтение и анализ логов бэнчмарка

# Импорты

In [37]:
import re
from pathlib import Path
from pprint import pprint
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from plotly import colors
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from src.utils.read_logs import LogReader

In [38]:
plt.ioff()
%matplotlib notebook

# Чтение логов

In [39]:
log_file_path = "logs/2025-03-10 method_logs.json"
logs = LogReader.load_logs_from_file(log_file_path)

In [40]:
len(logs)

10158

In [41]:
logs[0]

{'method_name': 'TensorLy_Tucker_image-0_pytorch_truncated_svd_svd',
 'method_args': {'rank': [252, 255, 3],
  'n_iter_max': 100,
  'init': 'svd',
  'svd': 'truncated_svd',
  'random_state': 42},
 'qualitative_metrics': {'Language': 'Python',
  'Library': 'TensorLy',
  'TensorLy backend': 'pytorch',
  'Tensor type': 'Dense',
  'Data type': 'image-0',
  'Platform': 'CPU, GPU',
  'Decomposition method': 'Tucker'},
 'quantitative_metrics': {'gpu_allocated_memory_used_mb': [9.95263671875,
   0.0,
   0.0,
   0.0,
   0.0],
  'gpu_cached_memory_used_mb': [30.0, 6.0, 8.0, 26.0, 4.0],
  'ram_mem_used_mb': [6237.703125,
   6238.25,
   6238.25,
   6238.25390625,
   6238.25390625],
  'duration': [4.4722535610198975,
   2.7524054050445557,
   1.7934534549713135,
   1.7476089000701904,
   2.4861092567443848],
  'frobenius_error': [0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604,
   0.6721850484609604],
  'compression_ratio': [50.166930737890446,
   50.16693073

In [48]:
clean_rows, anom_rows, error_rows = [], [], []
neg_stats = defaultdict(lambda: {"idx": [], "vals": []})        # статистика по аномалиям

metric_keys = [                      # порядок важен для синхронного среза
    "gpu_allocated_memory_used_mb",
    "gpu_cached_memory_used_mb",
    "ram_mem_used_mb",
    "duration",
    "frobenius_error",
    "compression_ratio",
]

for log in logs:
    # --- базовая (qualitative) часть ---------------------------------
    base = {
        "method_name"         : log["method_name"],
        "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
        "data_type"           : log["qualitative_metrics"]["Data type"],
        "language"            : log["qualitative_metrics"]["Language"],
        "library"             : log["qualitative_metrics"]["Library"],
        "tensor_type"         : log["qualitative_metrics"]["Tensor type"],
        "platform"            : log["qualitative_metrics"]["Platform"],
        "backend"             : log["qualitative_metrics"][
            next(k for k in log["qualitative_metrics"] if "backend" in k)
        ],
    }

    # ----- случаи с ошибкой выполнения --------------------------------
    if log["error_message"]:
        error_rows.append({**base, "error_message": log["error_message"]})
        continue

    q = log["quantitative_metrics"]
    n_runs = max(len(q[k]) for k in metric_keys)    # обычно 5

    # 1. определяем, какие индексы аномальны
    bad_idx = {i for k in metric_keys
               for i, v in enumerate(q[k])
               if isinstance(v, (int, float)) and v < 0}

    # 2. «чистые» и «плохие» индексы
    good_idx = sorted(set(range(n_runs)) - bad_idx)
    bad_idx  = sorted(bad_idx)

    # функция-хелпер: формирует агрегаты для выбранного подмножества индексов
    def make_row(idx_subset, tag):
        if not idx_subset:
            return None                     # нечего добавлять
        row = {**base,
               "run_type": tag,             # clean | anomaly
               "original_run_indices" : idx_subset}      # список позиций
        for k in metric_keys:
            vals = [q[k][i] for i in idx_subset]
            row[f"{k}_min"]  = np.min(vals)
            row[f"{k}_max"]  = np.max(vals)
            row[f"{k}_mean"] = np.mean(vals)
        return row

    # 3. сохраняем строки
    r_clean = make_row(good_idx,  "clean")
    r_anom  = make_row(bad_idx ,  "anomaly")

    if r_clean: clean_rows.append(r_clean)
    if r_anom:
        anom_rows.append(r_anom)
        # для общей статистики: сколько раз каждая позиция «проваливается»
        for k in metric_keys:
            neg_stats[k]['idx'].extend([i for i in bad_idx
                                 if isinstance(q[k][i], (int, float)) and q[k][i] < 0])
            neg_stats[k]['vals'].extend([q[k][i] for i in bad_idx
                                        if isinstance(q[k][i], (int, float)) and q[k][i] < 0])

# --- превращаем во фреймы --------------------------------------------
df_clean = pd.DataFrame(clean_rows)
df_anom  = pd.DataFrame(anom_rows)
df_error = pd.DataFrame(error_rows)

In [49]:
print("metric | count_neg | run_idx_range | min | mean | max")
print("-"*60)

for metric, d in neg_stats.items():
    if not d["vals"]:
        continue

    idx_range = f"{min(d['idx'])}–{max(d['idx'])}"
    print(f"{metric:<30} | "
          f"{len(d['vals']):>9} | "
          f"{idx_range:^13} | "
          f"{min(d['vals']):>6.4f} | "
          f"{np.mean(d['vals']):>6.4f} | "
          f"{max(d['vals']):>6.4f}")

metric | count_neg | run_idx_range | min | mean | max
------------------------------------------------------------
gpu_allocated_memory_used_mb   |      1516 |      1–4      | -43.6816 | -0.5823 | -0.0068
duration                       |        68 |      0–4      | -1.0781 | -0.4767 | -0.0128


In [42]:
log_data = []
log_errors = []

negative_stats = defaultdict(lambda: {
    "count": 0,
    "min": float("inf"),
    "max": float("-inf"),
    "sum": 0.0
})

for log in logs:
    if log["error_message"] == "":
        try:
            data_entry = {
                # Method name
                "method_name": log["method_name"],
                # Some Qualitative Data
                "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
                "data_type": log["qualitative_metrics"]["Data type"],
                "language": log["qualitative_metrics"]["Language"],
                "library": log["qualitative_metrics"]["Library"],
                "tensor_type": log["qualitative_metrics"]["Tensor type"],
                "platform": log["qualitative_metrics"]["Platform"],
                "backend": log["qualitative_metrics"].get(
                    next(key for key in log["qualitative_metrics"] if "backend" in key)
                ),
                # GPU Allocated Memory
                "gpu_allocated_memory_used_min": np.min(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used_max": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_allocated_memory_used": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_allocated_memory_used_mb"] if e >= 0.0]
                ),
                # GPU Cached Memory
                "gpu_cached_memory_used_min": np.min(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used_max": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                "gpu_cached_memory_used": np.max(
                    [e for e in log["quantitative_metrics"]["gpu_cached_memory_used_mb"] if e >= 0.0]
                ),
                # RAM Memory Usage
                "ram_mem_used_min": np.min([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used_max": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used_mean": np.mean([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                "ram_mem_used": np.max([e for e in log["quantitative_metrics"]["ram_mem_used_mb"] if e >= 0.0]),
                # Duration
                "duration_min": np.min([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration_max": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration_mean": np.mean([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                "duration": np.max([e for e in log["quantitative_metrics"]["duration"] if e >= 0.0]),
                # Frobenius Error
                "frobenius_error_min": np.min([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                "frobenius_error_max": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                "frobenius_error_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]
                ),
                "frobenius_error": np.max([e for e in log["quantitative_metrics"]["frobenius_error"] if e >= 0.0]),
                # Compression Ratio
                "compression_ratio_min": np.min(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio_max": np.max(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio_mean": np.mean(
                    [e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]
                ),
                "compression_ratio": np.max([e for e in log["quantitative_metrics"]["compression_ratio"] if e >= 0.0]),
            }
            log_data.append(data_entry)

            for key, values in log["quantitative_metrics"].items():
                if not isinstance(values, list):
                    continue

                neg_values = [v for v in values if isinstance(v, (int, float)) and v < 0]
                if neg_values:
                    print(f"[{log['method_name']}] Key '{key}' has negative values: {neg_values}")

                    negative_stats[key]["count"] += len(neg_values)
                    negative_stats[key]["min"] = min(negative_stats[key]["min"], min(neg_values))
                    negative_stats[key]["max"] = max(negative_stats[key]["max"], max(neg_values))
                    negative_stats[key]["sum"] += sum(neg_values)


        except Exception as e:
            print(f"Exception with {log['method_name']}. Error: {e!s}")
    else:
        data_entry = {
            # method name
            "method_name": log["method_name"],
            # qualitative metrics
            "decomposition_method": log["qualitative_metrics"]["Decomposition method"],
            "data_type": log["qualitative_metrics"]["Data type"],
            "language": log["qualitative_metrics"]["Language"],
            "library": log["qualitative_metrics"]["Library"],
            "tensor_type": log["qualitative_metrics"]["Tensor type"],
            "platform": log["qualitative_metrics"]["Platform"],
            "backend": log["qualitative_metrics"].get(
                next(key for key in log["qualitative_metrics"] if "backend" in key)
            ),
            # Error data
            "error_message": log["error_message"],
        }
        log_errors.append(data_entry)

df_logs = pd.DataFrame(log_data)
df_logs_errors = pd.DataFrame(log_errors)

[TensorLy_Tucker_image-2_pytorch_truncated_svd_svd] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.43603515625, -0.6103515625]
[TensorLy_Tucker_image-2_pytorch_symeig_svd_random] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.30908203125]
[TensorLy_Tucker_image-2_pytorch_randomized_svd_svd] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.11328125, -0.341796875, -0.33837890625]
[TensorLy_Tucker_image-2_pytorch_randomized_svd_random] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.01171875, -0.0322265625, -0.0791015625]
[TensorLy_Tucker_video-1_pytorch_truncated_svd_svd] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.23046875, -0.50390625]
[TensorLy_Tucker_video-1_pytorch_truncated_svd_random] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.359375, -0.359375]
[TensorLy_Tucker_video-1_pytorch_symeig_svd_random] Key 'gpu_allocated_memory_used_mb' has negative values: [-0.359375, -0.375]
[TensorLy_Tucker_video-1_pyto

In [43]:
print("\n=== Summary of Negative Values in Quantitative Metrics ===")
for key, stats in negative_stats.items():
    count = stats["count"]
    if count > 0:
        avg = stats["sum"] / count
        print(f"Key: {key}")
        print(f"  → Count   : {count}")
        print(f"  → Min     : {stats['min']}")
        print(f"  → Max     : {stats['max']}")
        print(f"  → Mean    : {avg:.4f}\n")


=== Summary of Negative Values in Quantitative Metrics ===
Key: gpu_allocated_memory_used_mb
  → Count   : 1516
  → Min     : -43.681640625
  → Max     : -0.0068359375
  → Mean    : -0.5823

Key: duration
  → Count   : 68
  → Min     : -1.0781352519989014
  → Max     : -0.012799739837646484
  → Mean    : -0.4767



In [34]:
df_logs.shape

(4772, 32)

In [19]:
df_logs_errors.shape

(5386, 9)

## Обработка ошибок

In [20]:
def extract_memory_info(error_message):
    pattern = r"(\d+\.?\d*)\s*(\w+iB)"
    match = re.search(pattern, error_message)

    if match:
        value = float(match.group(1))  # Память
        unit = match.group(2)  # Тип памяти (единица измерения)

        # Конвертация в MiB
        conversion_factors = {
            "KiB": 1 / 1024,  # Килобайты -> Мегабайты
            "MiB": 1,  # Мегабайты -> Мегабайты
            "GiB": 1024,  # Гигабайты -> Мегабайты
            "TiB": 1024**2,  # Терабайты -> Мегабайты
            "PiB": 1024**3,  # Петабайты -> Мегабайты
            "EiB": 1024**4,  # Эксабайты -> Мегабайты
            "ZiB": 1024**5,  # Зеттабайты -> Мегабайты
            "YiB": 1024**6,  # Йоттабайты -> Мегабайты
        }

        if unit in conversion_factors:
            value_in_mib = value * conversion_factors[unit]
        else:
            raise ValueError(f"Неизвестная единица измерения памяти: {unit}")

        return value_in_mib, "MiB"

    return None, None

In [21]:
error_messages_count = 0

for index, row in df_logs_errors.iterrows():
    error_message = row["error_message"]

    if "Tried to allocate" in error_message:  # pytorch
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из GPU and RAM ошибки.")
    elif "Unable to allocate" in error_message:  # numpy
        value, unit = extract_memory_info(error_message)
        if not (value and unit):
            print("Не удалось извлечь информацию о памяти из RAM ошибки.")
    else:
        error_messages_count+=1
        print(error_message)

    df_logs_errors.at[index, "memory_tried_to_allocated_error_mib"] = value
print(error_messages_count)

Пропущена итерация из-за ошибки: pytorch, truncated_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, symeig_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, randomized_svd, random, False, False, 1e-08, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, truncated_svd, random, False, False, 1e-05, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, symeig_svd, random, False, False, 1e-05, 0, abs_rec_error. Ошибка: torch.linalg.solve: The solver failed because the input matrix is singular.
Пропущена итерация из-за ошибки: pytorch, randomized_svd, random,

In [22]:
df_logs_errors.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,memory_tried_to_allocated_error_mib
0,TensorLy_Tucker_image-0_pytorch_symeig_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",385996.80
1,TensorLy_Tucker_image-1_pytorch_symeig_svd_svd,Tucker,image-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",248903.68
2,TensorLy_Tucker_image-2_pytorch_symeig_svd_svd,Tucker,image-2,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",2586040.32
3,TensorLy_Tucker_video-0_pytorch_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",46653.44
4,TensorLy_Tucker_video-1_pytorch_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",26245.12


# Функции для анализа логов

In [23]:
# Отрисовка горизонтальных бар-графиков для метрик
def plot_barh(ax, x_data, y_data, title, xlabel, best_value=None, best_label=None, color="green"):
    ax.barh(y_data, x_data, color=color)
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    if best_value is not None and best_label is not None:
        ax.axvline(x=best_value, color="red", linestyle="--", label=best_label)
        ax.legend()
    for i in range(len(x_data)):
        ax.text(x_data.iloc[i], i, f"{x_data.iloc[i]:.6f}", va="center")

In [24]:
def get_metrics_data(filtered_df):
    filtered_df_unique_by_method_name = filtered_df["method_name"].unique()
    analysis_data = []

    for method in filtered_df_unique_by_method_name:
        method_specific_data = filtered_df[filtered_df["method_name"] == method]

        compression_ratio = method_specific_data["compression_ratio"].to_numpy()[0]
        duration = method_specific_data["duration"].to_numpy()[0]
        total_memory = (
            method_specific_data["ram_mem_used"].to_numpy()[0]
            + method_specific_data["gpu_cached_memory_used"].to_numpy()[0]
        )
        frobenius_error = method_specific_data["frobenius_error"].to_numpy()[0]

        analysis_data.append(
            {
                "method_name": method,
                "duration": duration,
                "total_memory": total_memory,
                "frobenius_error": frobenius_error,
                "compression_ratio": compression_ratio,
            }
        )

    return pd.DataFrame(analysis_data)

In [25]:
# Анализ и визуализация методов по количественным показателям в разрезе качественного показателя
def plot_barhs_and_analyze_dfs(df_logs, group_name: str = "decomposition_method"):
    plt.rcParams.update(
        {
            "axes.titlesize": 16,
            "axes.labelsize": 14,
            "xtick.labelsize": 12,
            "ytick.labelsize": 12,
            "legend.fontsize": 12,
        }
    )
    groups = df_logs[group_name].unique()
    analysis_results = {}

    for group in groups:
        filtered_df = df_logs[df_logs[group_name] == group]

        analysis_df = get_metrics_data(filtered_df)

        best_time_method = analysis_df.loc[analysis_df["duration"].idxmin()]
        best_memory_method = analysis_df.loc[analysis_df["total_memory"].idxmin()]
        best_error_method = analysis_df.loc[analysis_df["frobenius_error"].idxmin()]

        analysis_results[group] = {
            "best_time_method": best_time_method.to_dict(),
            "best_memory_method": best_memory_method.to_dict(),
            "best_error_method": best_error_method.to_dict(),
        }
    #
    #     num_methods = len(analysis_df["method_name"])
    #     height_per_method = 0.5
    #     fig_height = max(36, num_methods * height_per_method)
    #     fig_height = 2 ** 16 - 1 if 2 ** 16 < fig_height else fig_height
    #
    #     fig, axes = plt.subplots(4, 1, figsize=(12, fig_height))
    #
    #     plot_barh(
    #         axes[0],
    #         analysis_df["compression_ratio"],
    #         analysis_df["method_name"],
    #         f"Сравнение сжатия по {group_name} - {group}",
    #         "Сжатие (%)",
    #     )
    #
    #     plot_barh(
    #         axes[1],
    #         analysis_df["duration"],
    #         analysis_df["method_name"],
    #         f"Время исполнения по {group_name} - {group}",
    #         "Время (с)",
    #         best_time_method["duration"],
    #         "Лучший метод",
    #         color="skyblue",
    #     )
    #
    #     plot_barh(
    #         axes[2],
    #         analysis_df["total_memory"],
    #         analysis_df["method_name"],
    #         f"Сумма затрачиваемой памяти по {group_name} - {group}",
    #         "Память (МБ)",
    #         best_memory_method["total_memory"],
    #         "Лучший метод",
    #         color="lightgreen",
    #     )
    #
    #     plot_barh(
    #         axes[3],
    #         analysis_df["frobenius_error"],
    #         analysis_df["method_name"],
    #         f"Ошибка Фробениуса по {group_name} - {group}",
    #         "Ошибка (%)",
    #         best_error_method["frobenius_error"],
    #         "Лучший метод",
    #         color="salmon",
    #     )
    #
    #     plt.tight_layout()
    #     plt.subplots_adjust(top=0.95, bottom=0.05, left=0.05, right=0.95)
    #     plt.show()

    return analysis_results

In [26]:
# # Отрисовка горизонтальных бар-графиков для метрик
# def plot_error_bar(ax, data_mean, data_min, data_max, title, ylabel, label):
#     yerr_lower = data_mean - data_min
#     yerr_upper = data_max - data_mean
#
#     yerr_lower = max(yerr_lower, 0)
#     yerr_upper = max(yerr_upper, 0)
#
#     ax.errorbar(
#         [0], [data_mean], yerr=[[yerr_lower], [yerr_upper]], fmt="o",
#         markersize=10, capsize=10, capthick=3, elinewidth=3, color="lightgreen", label=label
#     )
#     ax.set_title(title)
#     ax.set_ylabel(ylabel)
#     ax.set_xticks([0])
#     ax.set_xticklabels([label])
#
#     for val, _lbl in zip([data_mean, data_min, data_max], ["Mean", "Min", "Max"], strict=False):
#         ax.text(0, val, f"{val:.2f}", ha="center", va="bottom", fontsize=10, fontweight="bold")
#
#     ax.axhline(y=data_mean, color="grey", linestyle="--")
#     ax.axhline(y=data_min, color="lightgrey", linestyle=":")
#     ax.axhline(y=data_max, color="lightgrey", linestyle=":")

In [27]:
# # Анализ и визуализация методов по количественным показателям по минимальным, максимальным и средним значениям
# def plot_error_bars(method_data, method):
#     fig, axes = plt.subplots(3, 2, figsize=(16, 16))
#     fig.suptitle(f"Графики для метода: {method}", fontsize=16, fontweight="bold", color="green")
#
#     plot_error_bar(
#         axes[0, 0],
#         method_data["gpu_allocated_memory_used"].to_numpy()[0],
#         method_data["gpu_allocated_memory_used_min"].to_numpy()[0],
#         method_data["gpu_allocated_memory_used_max"].to_numpy()[0],
#         "GPU Allocated Memory",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[0, 1],
#         method_data["gpu_cached_memory_used"].to_numpy()[0],
#         method_data["gpu_cached_memory_used_min"].to_numpy()[0],
#         method_data["gpu_cached_memory_used_max"].to_numpy()[0],
#         "GPU Cached Memory",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[1, 0],
#         method_data["ram_mem_used"].to_numpy()[0],
#         method_data["ram_mem_used_min"].to_numpy()[0],
#         method_data["ram_mem_used_max"].to_numpy()[0],
#         "RAM Memory Usage",
#         "Memory (MB)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[1, 1],
#         method_data["duration"].to_numpy()[0],
#         method_data["duration_min"].to_numpy()[0],
#         method_data["duration_max"].to_numpy()[0],
#         "Duration",
#         "Time (s)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[2, 0],
#         method_data["frobenius_error"].to_numpy()[0],
#         method_data["frobenius_error_min"].to_numpy()[0],
#         method_data["frobenius_error_max"].to_numpy()[0],
#         "Frobenius Error",
#         "Error (%)",
#         method,
#     )
#
#     plot_error_bar(
#         axes[2, 1],
#         method_data["compression_ratio"].to_numpy()[0],
#         method_data["compression_ratio_min"].to_numpy()[0],
#         method_data["compression_ratio_max"].to_numpy()[0],
#         "Compression Ratio",
#         "Ratio",
#         method,
#     )
#
#     plt.tight_layout(rect=[0, 0, 1, 0.95])
#     plt.show()

In [35]:
df_logs.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,gpu_allocated_memory_used_min,gpu_allocated_memory_used_max,...,duration_mean,duration,frobenius_error_min,frobenius_error_max,frobenius_error_mean,frobenius_error,compression_ratio_min,compression_ratio_max,compression_ratio_mean,compression_ratio
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,9.952637,...,2.650366,4.472254,0.672185,0.672185,0.672185,0.672185,50.166931,50.166931,50.166931,50.166931
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.577740,2.642248,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.695470,2.954981,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
3,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,1.336961,1.372077,0.672679,0.672679,0.672679,0.672679,50.166931,50.166931,50.166931,50.166931
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.675934,2.859465,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931


# Расчет лучших метрик

## Сравнение методов по типам данных

In [28]:
analysis_results_data_type = plot_barhs_and_analyze_dfs(df_logs, group_name="data_type")

## Сравнение метрик по методам

In [29]:
analysis_results_decomposition_method = plot_barhs_and_analyze_dfs(df_logs, group_name="decomposition_method")

## Сравнение метрик по методам и типам данных

In [30]:
data_types = df_logs["data_type"].unique()

df_logs_by_data_types_dict = {data_type: df_logs[df_logs["data_type"] == data_type] for data_type in data_types}

analysis_results_data_types_decompositions_methods = {
    data_type: plot_barhs_and_analyze_dfs(df_logs_by_data_types_dict[data_type], group_name="decomposition_method")
    for data_type in data_types
}

# Сравнение метрик

## Метрики в разрезе типов данных

In [31]:
pprint(analysis_results_data_type, indent=4)

{   'eeg-0': {   'best_error_method': {   'compression_ratio': 52.237848577500216,
                                          'duration': 18.67386269569397,
                                          'frobenius_error': 0.025881556211970747,
                                          'method_name': 'TensorLy_TensorTrain_eeg-0_pytorch_randomized_svd',
                                          'total_memory': 5738.609375},
                 'best_memory_method': {   'compression_ratio': 52.237848577500216,
                                           'duration': 2.4023776054382324,
                                           'frobenius_error': 0.02617962018121034,
                                           'method_name': 'TensorLy_TensorTrain_eeg-0_pytorch_truncated_svd',
                                           'total_memory': 5239.2421875},
                 'best_time_method': {   'compression_ratio': 52.237848577500216,
                                         'duration': 2.4023776054382324

## Метрики в разрезе методов

In [32]:
pprint(analysis_results_decomposition_method, indent=4)

{   'CP': {   'best_error_method': {   'compression_ratio': 50.06507925904784,
                                       'duration': 0.8409609794616699,
                                       'frobenius_error': 0.06767631857655942,
                                       'method_name': 'TensorLy_CP_image-2_pytorch_truncated_svd_random_False_True_abs_rec_error_1.0_1e-08',
                                       'total_memory': 8697.734375},
              'best_memory_method': {   'compression_ratio': 50.05868249417367,
                                        'duration': 0.66693115234375,
                                        'frobenius_error': 0.4365097265690565,
                                        'method_name': 'TensorLy_CP_image-0_pytorch_symeig_svd_random_False_False_abs_rec_error_0_1e-05',
                                        'total_memory': 5508.3828125},
              'best_time_method': {   'compression_ratio': 50.020879006159305,
                                      'durat

## Метрики в разрезе типов данных и методов

In [33]:
for data_type, metrics in analysis_results_data_types_decompositions_methods.items():
    print(f"\n{data_type}\n")
    pprint(metrics, indent=4)


image-0

{   'CP': {   'best_error_method': {   'compression_ratio': 50.05868249417367,
                                       'duration': 0.8215165138244629,
                                       'frobenius_error': 0.3912588581442833,
                                       'method_name': 'TensorLy_CP_image-0_pytorch_truncated_svd_random_False_True_abs_rec_error_0.01_1e-08',
                                       'total_memory': 7370.28515625},
              'best_memory_method': {   'compression_ratio': 50.05868249417367,
                                        'duration': 0.66693115234375,
                                        'frobenius_error': 0.4365097265690565,
                                        'method_name': 'TensorLy_CP_image-0_pytorch_symeig_svd_random_False_False_abs_rec_error_0_1e-05',
                                        'total_memory': 5508.3828125},
              'best_time_method': {   'compression_ratio': 50.05868249417367,
                                 

# Аналитика логов количественных и качественных метрик

## Обработка данных

In [28]:
df_logs.shape

(4772, 32)

In [29]:
df_logs.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,gpu_allocated_memory_used_min,gpu_allocated_memory_used_max,...,duration_mean,duration,frobenius_error_min,frobenius_error_max,frobenius_error_mean,frobenius_error,compression_ratio_min,compression_ratio_max,compression_ratio_mean,compression_ratio
0,TensorLy_Tucker_image-0_pytorch_truncated_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,9.952637,...,2.650366,4.472254,0.672185,0.672185,0.672185,0.672185,50.166931,50.166931,50.166931,50.166931
1,TensorLy_Tucker_image-0_pytorch_truncated_svd_...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.577740,2.642248,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
2,TensorLy_Tucker_image-0_pytorch_symeig_svd_random,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.695470,2.954981,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931
3,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,1.336961,1.372077,0.672679,0.672679,0.672679,0.672679,50.166931,50.166931,50.166931,50.166931
4,TensorLy_Tucker_image-0_pytorch_randomized_svd...,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,0.0,1.827637,...,2.675934,2.859465,0.672281,0.672281,0.672281,0.672281,50.166931,50.166931,50.166931,50.166931


In [30]:
df_filtered_for_analytics = df_logs[
    [
        "method_name",
        "data_type",
        "backend",
        "decomposition_method",
        "gpu_allocated_memory_used",
        "gpu_allocated_memory_used_mean",
        "gpu_cached_memory_used",
        "gpu_cached_memory_used_mean",
        "ram_mem_used",
        "ram_mem_used_mean",
        "duration",
        "duration_mean",
        "frobenius_error",
        "frobenius_error_mean",
        "compression_ratio",
        "compression_ratio_mean",
    ]
]

In [31]:
dfs_by_decomposition_method = {
    decomposition_method: df_filtered_for_analytics[
        df_filtered_for_analytics["decomposition_method"] == decomposition_method
    ]
    for decomposition_method in df_filtered_for_analytics["decomposition_method"].unique()
}

In [32]:
for decomposition_method, df_by_decomposition_method in dfs_by_decomposition_method.items():
    enriched_rows = []
    for _, row in df_by_decomposition_method.iterrows():
        matching_log = next((log for log in logs if row["method_name"] == log["method_name"]), None)

        enriched_row = row.copy()

        if row["decomposition_method"] == "Tucker":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "TensorTrain":
            enriched_row["svd"] = matching_log["method_args"].get("svd")
        elif row["decomposition_method"] == "CP":
            enriched_row["init"] = matching_log["method_args"].get("init")
            enriched_row["svd"] = matching_log["method_args"].get("svd")
            enriched_row["normalize_factors"] = matching_log["method_args"].get("normalize_factors")
            enriched_row["orthogonalise"] = matching_log["method_args"].get("orthogonalise")
            enriched_row["tol"] = matching_log["method_args"].get("tol")
            enriched_row["l2_reg"] = matching_log["method_args"].get("l2_reg")
            enriched_row["cvg_criterion"] = matching_log["method_args"].get("cvg_criterion")

        enriched_rows.append(enriched_row)

    dfs_by_decomposition_method[decomposition_method] = pd.DataFrame(enriched_rows)

In [33]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Обработка ошибок

In [34]:
df_logs_errors.shape

(5386, 10)

In [35]:
df_logs_errors.head()

,method_name,decomposition_method,data_type,language,library,tensor_type,platform,backend,error_message,memory_tried_to_allocated_error_mib
0,TensorLy_Tucker_image-0_pytorch_symeig_svd_svd,Tucker,image-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",385996.80
1,TensorLy_Tucker_image-1_pytorch_symeig_svd_svd,Tucker,image-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",248903.68
2,TensorLy_Tucker_image-2_pytorch_symeig_svd_svd,Tucker,image-2,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",2586040.32
3,TensorLy_Tucker_video-0_pytorch_symeig_svd_svd,Tucker,video-0,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",46653.44
4,TensorLy_Tucker_video-1_pytorch_symeig_svd_svd,Tucker,video-1,Python,TensorLy,Dense,"CPU, GPU",pytorch,"Пропущена итерация из-за ошибки: pytorch, syme...",26245.12


## Методы для графиков

In [28]:
def preprocess_dataframe(df):
    df_encoded = df.copy()
    for column in df_encoded.columns:
        if df_encoded[column].dtype == "object" and column != "method_name":
            df_encoded[column] = LabelEncoder().fit_transform(df_encoded[column])

    scaler = MinMaxScaler()
    df_encoded[df_encoded.columns.difference(["method_name"])] = scaler.fit_transform(
        df_encoded[df_encoded.columns.difference(["method_name"])]
    )
    return df_encoded

In [29]:
def plot_heatmap(df, title):
    folder_path = Path("../.cache/data_analyze/")

    corr_matrix = df.drop(columns=["method_name"]).corr()

    fig = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale=colors.sequential.RdBu,
            colorbar={
                "title": "Correlation",
                "titlefont": {"color": "black"},
                "tickfont": {"color": "black"},
            },
            text=corr_matrix.values,
            texttemplate="%{text:.2f}",
            textfont={"size": 12, "color": "black"},
            hovertemplate="<b>%{x}</b> vs. <b>%{y}</b> <b>Correlation:</b> %{z:.8f}<extra></extra>",
        )
    )

    fig.update_layout(
        title=f"Heatmap: {title}",
        title_font={"color": "black"},
        xaxis={"tickangle": -45, "tickfont": {"size": 12, "color": "black"}},
        yaxis={"tickfont": {"size": 12, "color": "black"}},
        width=1400,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    folder_path.mkdir(parents=True, exist_ok=True)

    fig.write_html(folder_path / f"heatmap_{title}.html")

In [30]:
def plot_pca(df, title):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pca_df = df.drop(columns=["method_name"])
    pca = PCA(n_components=3)
    pca_components = pca.fit_transform(pca_df)

    pca_df = pd.DataFrame(
        {
            "PCA1": pca_components[:, 0],
            "PCA2": pca_components[:, 1],
            "PCA3": pca_components[:, 2],
            "method_name": df["method_name"].to_numpy(),
        }
    )

    fig1 = px.scatter(
        pca_df, x="PCA1", y="PCA2", color="method_name", title=f"PCA 2D Projection: {title}", hover_name="method_name"
    )
    fig1.update_traces(marker={"size": 10})
    fig1.update_layout(
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        xaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        yaxis={
            "titlefont": {"size": 12, "color": "black"},
            "tickfont": {"size": 12, "color": "black"},
        },
        showlegend=False,
    )

    fig1.write_html(folder_path / f"pca_2d_{title}.html")

    fig2 = px.scatter_3d(
        pca_df,
        x="PCA1",
        y="PCA2",
        z="PCA3",
        color="method_name",
        title=f"PCA 3D Projection: {title}",
        hover_name="method_name",
    )
    fig2.update_traces(marker={"size": 5})
    fig2.update_layout(
        scene={
            "xaxis_title": "PCA1",
            "yaxis_title": "PCA2",
            "zaxis_title": "PCA3",
            "xaxis_backgroundcolor": "white",
            "yaxis_backgroundcolor": "white",
            "zaxis_backgroundcolor": "white",
            "xaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "yaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "zaxis": {
                "titlefont": {"size": 12, "color": "black"},
                "tickfont": {"size": 12, "color": "black"},
            },
            "bgcolor": "white",
        },
        width=1800,
        height=1400,
        plot_bgcolor="white",
        paper_bgcolor="white",
        title_font={"color": "black"},
        showlegend=False,
    )

    fig2.write_html(folder_path / f"pca_3d_{title}.html")

In [31]:
def plot_pairplot(df, title, sample_size=1000, num_axes=100):
    folder_path = Path("../.cache/data_analyze/")
    folder_path.mkdir(parents=True, exist_ok=True)

    pairplot_df = df.drop(columns=["method_name"])

    if len(pairplot_df) > sample_size:
        pairplot_df = pairplot_df.sample(n=sample_size, random_state=42)

    selected_axes = pairplot_df.columns[:num_axes]

    sns.set(style="white")
    g = sns.pairplot(pairplot_df[selected_axes], diag_kind="kde", markers="+", plot_kws={"s": 50})
    g.fig.suptitle(f"Pairplot: {title}", y=1.02, fontsize=16, color="black")

    for ax in g.axes.flatten():
        ax.tick_params(axis="x", labelsize=12, labelcolor="black")
        ax.tick_params(axis="y", labelsize=12, labelcolor="black")
        ax.set_facecolor("white")
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

    plt.savefig(folder_path / f"pairplot_{title}.png", dpi=300, bbox_inches="tight")
    plt.close("all")

In [32]:
dfs_by_decomposition_method_encoded = {}

In [33]:
dfs_by_decomposition_method.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

In [34]:
for key, df_by_decomposition_method in dfs_by_decomposition_method.items():
    dfs_by_decomposition_method_encoded[key] = preprocess_dataframe(
        df_by_decomposition_method.drop("decomposition_method", axis=1)
    )

In [35]:
# dfs_by_decomposition_method['CP']

In [36]:
# dfs_by_decomposition_method_encoded['CP']

In [37]:
dfs_by_decomposition_method_encoded.keys()

dict_keys(['Tucker', 'TensorTrain', 'CP'])

## Heatmap

In [38]:
method_name = "Tucker"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [39]:
method_name = "TensorTrain"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [40]:
method_name = "CP"
plot_heatmap(dfs_by_decomposition_method_encoded[method_name], title=method_name)

## PCA

In [41]:
method_name = "Tucker"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [42]:
method_name = "TensorTrain"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

In [43]:
method_name = "CP"
plot_pca(dfs_by_decomposition_method_encoded[method_name], title=method_name)

## Pairplot

In [44]:
method_name = "Tucker"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

In [45]:
method_name = "TensorTrain"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

In [46]:
method_name = "CP"
plot_pairplot(dfs_by_decomposition_method_encoded[method_name], title=method_name)

<IPython.core.display.Javascript object>

# Аналитика логов ошибок